In [37]:
##Data Processing and Cleaning:


import pandas as pd
# Read the CSV file
df = pd.read_csv("Saudi_Arabia_hotels.csv", names=["Hotel Name", "Address", "Score", "Number of reviews", "Price", "Room type", "Star rating", "Room view", "Swimming pool", "Restaurant", "Parking", "Wheelchair accessibility"])
df = df.drop(index=0)
df



,Hotel Name,Address,Score,Number of reviews,Price,Room type,Star rating,Room view,Swimming pool,Restaurant,Parking,Wheelchair accessibility
1,Malfa Hotel Apartments,Qal'at Bishah,8.1Scored 8.1,823 reviews,"SAR 1,250",Twin Room with Private Bathroom,NaN,yes,no,no,yes,yes
2,Intour Al Khafji Hotel,Al Khafji,7.7Scored 7.7,474 reviews,"SAR 1,465",Deluxe Room,4.0,no,no,yes,yes,no
3,Braira Al Ahsa,Al Ahsa,8.9Scored 8.9,"2,750 reviews","SAR 3,071",Classic Room,4.0,yes,yes,yes,yes,no
4,الراحة بلازا للشقق المفروشة,Sharurah,7.1Scored 7.1,96 reviews,SAR 625,Twin Room,3.0,yes,no,no,yes,no
5,Boudl Majmaa,Al Majmaah,8.1Scored 8.1,700 reviews,"SAR 1,656",Standard Room With King Bed,NaN,no,yes,yes,yes,no
...,...,...,...,...,...,...,...,...,...,...,...,...
671,Raoum Inn Khafji Corniche,Al Khafji,8.3Scored 8.3,352 reviews,"SAR 1,489",Double or Twin Room with Side Sea View,NaN,yes,yes,yes,yes,yes
672,Ramada Hotel & Suites by Wyndham Al Qassim,Al Bukayriyah,8.9Scored 8.9,"1,352 reviews","SAR 1,743",King Room - Smoking,5.0,yes,yes,yes,yes,yes
673,Makarim Palm Hotel,Rabigh,6.9Scored 6.9,240 reviews,"SAR 1,013",One-Bedroom Apartment,NaN,yes,no,no,yes,no
674,Malfa Hotel Apartments,Qal'at Bishah,8.1Scored 8.1,823 reviews,"SAR 1,250",Twin Room with Private Bathroom,NaN,yes,no,no,yes,yes


In [38]:
# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Display the count of missing values for each column
print("Missing values:\n", missing_values)


Missing values:
 Hotel Name                    0
Address                       0
Score                         3
Number of reviews             3
Price                         0
Room type                     0
Star rating                 324
Room view                     0
Swimming pool                 0
Restaurant                    0
Parking                       0
Wheelchair accessibility      0
dtype: int64


Since there are only a few missing values for Score and Number of reviews we consider filling them in with the mean.
For the Star rating Since there are quite a few missing values for this column (324), filling them in might not be appropriate. Instead we consider that it's appropriate to replace them with NaN.

In [39]:
import numpy as np
# Convert the 'Score' column to string type
df['Score'] = df['Score'].astype(str)

# Remove "Scored" from the values in the 'Score' column
df['Score'] = df['Score'].str.replace('Scored', '')

# Split the values by space and take the first score
df['Score'] = df['Score'].str.split().str[0]

# Replace "nan" with NaN
df['Score'] = df['Score'].replace('nan', np.nan)

# Convert the 'Score' column to numeric
df['Score'] = pd.to_numeric(df['Score'])

# Calculate the mean score excluding NaN values
mean_score = df['Score'].mean()

# Replace missing values in 'Score' column with the mean
df['Score'].fillna(mean_score, inplace=True)



we find diffuculty when fiilinf the score with the mean since each score is followed by (Scored), so we replace this word with "", then we convert the values into numbers after that we calculate the mean of the socre 

In [40]:
# Convert the 'Number of reviews' column to string and then replace "( reviews)" text
df['Number of reviews'] = df['Number of reviews'].astype(str).str.replace(' reviews', '')

# Convert the 'Number of reviews' column to integer type
df['Number of reviews'] = pd.to_numeric(df['Number of reviews'], errors='coerce').fillna(0).astype(int)
# Calculate the mean number of reviews excluding NaN values
mean_reviews = df['Number of reviews'].mean()

# Replace missing values in 'Number of reviews' column with the mean
df['Number of reviews'].fillna(mean_reviews, inplace=True)





In [41]:
#Replace NaN values in 'Star rating' column with zero
df['Star rating'].fillna(0, inplace=True)


In [66]:
# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Display the count of missing values for each column
print("Missing values:\n", missing_values)

# Assuming df is your DataFrame with fixed missing values
df.to_csv('mod_KSA_hotels.csv', index=False)


Missing values:
 Hotel Name                  0
Address                     0
Score                       0
Number of reviews           0
Price                       0
Room type                   0
Star rating                 0
Room view                   0
Swimming pool               0
Restaurant                  0
Parking                     0
Wheelchair accessibility    0
dtype: int64


Missing values were addressed and the format of 'Score' and 'Number of reviews' columns standardized. Initially, text ("Scored" and "reviews") was removed from respective columns, followed by conversion to numeric type for 'Score' and integer type for 'Number of reviews'. Missing values were then filled with the mean value of each column, excluding NaN values. These steps ensured data consistency and prepared it for analysis.

1.2 Evaluate the relevance of features and remove any irrelevant ones. 

In [43]:
    # To check if hotel is relevant with pools
    KSA_hotels = pd.read_csv('Saudi_Arabia_hotels.csv')
    pool_hotel = pd.read_excel('hotels_with_pool.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    pool_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    pool_attributes_hotel_ratings = KSA_hotels[hotels]
    hotel_pool = pd.merge(pool_attributes_hotel_ratings, pool_hotel, on='Hotel Name', how='inner')

    hotel_pool = hotel_pool.drop_duplicates()
    hotel_pool.to_csv('hotel_pool.csv', index=False)
    print(hotel_pool)

                                            Hotel Name          Score  \
0                                       Braira Al Ahsa  8.9Scored 8.9   
14                                        Boudl Majmaa  8.1Scored 8.1   
196                                    Boudl Al Jubail  8.4Scored 8.4   
197                            Warwick Al Jubail Hotel  8.6Scored 8.6   
198                             فندق كوخ الضباب النماص  7.7Scored 7.7   
...                                                ...            ...   
1776                                   Rio Hotel Olaya  8.0Scored 8.0   
1777                       فندق سافوي جدة Savoye Hotel  8.9Scored 8.9   
1778                         Crown Rose AlSahafa Hotel  7.4Scored 7.4   
1779               Rest Night Hotel Apartment- AlHamra  7.5Scored 7.5   
1780  Radisson Hotel & Apartments Dammam Industry City  7.9Scored 7.9   

      Star rating  
0             4.0  
14            NaN  
196           4.0  
197           4.0  
198           4.0  
...

In [44]:
    # To check if hotel is relevant with restaurants
    KSA_hotels = pd.read_csv('Saudi_Arabia_hotels.csv')
    restaurant_hotel = pd.read_excel('hotels_with_restaurant.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    restaurant_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    restaurant_hotel_ratings = KSA_hotels[hotels]
    restaurant_hotel = pd.merge(restaurant_hotel_ratings, restaurant_hotel, on='Hotel Name', how='inner')

    restaurant_hotel = restaurant_hotel.drop_duplicates()
    restaurant_hotel.to_csv('restaurant_hotel.csv', index=False)
    print(restaurant_hotel)

                                            Hotel Name          Score  \
0                               Intour Al Khafji Hotel  7.7Scored 7.7   
12                                      Braira Al Ahsa  8.9Scored 8.9   
23                                        Boudl Majmaa  8.1Scored 8.1   
192                           Caravan by Habitas AlUla  9.2Scored 9.2   
204                       AlMuhaidb Residence Alkhafji  8.4Scored 8.4   
...                                                ...            ...   
1802              M Hotel Al Dana Makkah by Millennium  7.9Scored 7.9   
1804                            Sama Al Qasr - Khurais  7.1Scored 7.1   
1807                          La Fontaine Jeddah Hotel  7.3Scored 7.3   
1809                         Crown Rose AlSahafa Hotel  7.4Scored 7.4   
1810  Radisson Hotel & Apartments Dammam Industry City  7.9Scored 7.9   

      Star rating  
0             4.0  
12            4.0  
23            NaN  
192           5.0  
204           NaN  
...

In [45]:
    # To check if hotel is relevant with parking
    KSA_hotels = pd.read_csv('Saudi_Arabia_hotels.csv')
    parking_hotel = pd.read_excel('hotels_with_parking.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    parking_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    parking_hotel_ratings = KSA_hotels[hotels]
    parking_hotel = pd.merge(parking_hotel_ratings, parking_hotel, on='Hotel Name', how='inner')

    parking_hotel = parking_hotel.drop_duplicates()
    parking_hotel.to_csv('parking_hotel.csv', index=False)
    print(parking_hotel)

                                          Hotel Name          Score  \
0                             Malfa Hotel Apartments  8.1Scored 8.1   
16                            Intour Al Khafji Hotel  7.7Scored 7.7   
24                                    Braira Al Ahsa  8.9Scored 8.9   
31                       الراحة بلازا للشقق المفروشة  7.1Scored 7.1   
32                                      Boudl Majmaa  8.1Scored 8.1   
...                                              ...            ...   
1468                                     Aster Hotel  7.5Scored 7.5   
1469  Holiday Inn Al Khobar - Corniche, an IHG Hotel  7.2Scored 7.2   
1470                     Courtyard by Marriott Jazan  8.5Scored 8.5   
1471                 Lotaz Hotel Suites - Al Salamah  9.1Scored 9.1   
1472                           Novotel Jeddah Tahlia  7.6Scored 7.6   

      Star rating  
0             NaN  
16            4.0  
24            4.0  
31            3.0  
32            NaN  
...           ...  
1468   

In [46]:
    # To check if hotel is relevant with wheelchair
    KSA_hotels = pd.read_csv('Saudi_Arabia_hotels.csv')
    wheelchair_hotel = pd.read_excel('hotels_with_wheelchair.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    wheelchair_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    wheelchair_hotel_ratings = KSA_hotels[hotels]
    wheelchair_hotel = pd.merge(wheelchair_hotel_ratings, wheelchair_hotel, on='Hotel Name', how='inner')

    wheelchair_hotel = wheelchair_hotel.drop_duplicates()
    wheelchair_hotel.to_csv('wheelchair_hotel.csv', index=False)
    print(wheelchair_hotel)

                            Hotel Name          Score  Star rating
0               Malfa Hotel Apartments  8.1Scored 8.1          NaN
16                      فندق نوفا بارك  9.2Scored 9.2          NaN
17                       Boudl Al Rass  8.7Scored 8.7          NaN
20              Lily Hotel Suite Hofuf  8.4Scored 8.4          NaN
42                 اقامة بلس ميرا هاوس  7.7Scored 7.7          3.0
...                                ...            ...          ...
1710  Residence Inn by Marriott Dammam  9.0Scored 9.0          4.0
1711      Courtyard by Marriott Jubail  8.6Scored 8.6          4.0
1712            Mövenpick Hotel Jeddah  7.6Scored 7.6          4.0
1713      Novotel Dammam Business Park  7.3Scored 7.3          4.0
1715   Traveller Inn Hotel Appartments  7.4Scored 7.4          NaN

[78 rows x 3 columns]


In [55]:
import pandas as pd

# Read the CSV file
KSA_hotels = pd.read_csv('mod_KSA_hotels.csv')

# Preprocessing 'Score' column into float 
KSA_hotels['Score'] = KSA_hotels['Score'].str.slice(9).astype(float)

# Preprocessing 'Number of reviews' column into int
KSA_hotels['Number of reviews'] = KSA_hotels['Number of reviews'].str.slice(stop=-7).str.replace(',','').astype(float)
KSA_hotels['Number of reviews'].fillna(0, inplace=True)  # Fill NaN values with 0
KSA_hotels['Number of reviews'] = KSA_hotels['Number of reviews'].astype(int)  # Convert to integer

# Preprocessing 'Price' column into int
KSA_hotels['Price'] = KSA_hotels['Price'].str.slice(3).str.replace(',','').astype(int)

print(KSA_hotels)


                                     Hotel Name        Address  Score  \
0                        Malfa Hotel Apartments  Qal'at Bishah    8.1   
1                        Intour Al Khafji Hotel      Al Khafji    7.7   
2                                Braira Al Ahsa        Al Ahsa    8.9   
3                   الراحة بلازا للشقق المفروشة       Sharurah    7.1   
4                                  Boudl Majmaa     Al Majmaah    8.1   
..                                          ...            ...    ...   
670                   Raoum Inn Khafji Corniche      Al Khafji    8.3   
671  Ramada Hotel & Suites by Wyndham Al Qassim  Al Bukayriyah    8.9   
672                          Makarim Palm Hotel         Rabigh    6.9   
673                      Malfa Hotel Apartments  Qal'at Bishah    8.1   
674                      Intour Al Khafji Hotel      Al Khafji    7.7   

     Number of reviews  Price                               Room type  \
0                  823   1250         Twin Room wi

In [48]:
# We noticed that the scores and star rating are not correlated to each other. Star ratings have many missing values. 
# Scores are the ratings given by the users themselves, therefore we chose to keep the scores and remove ratings

column_to_drop = 'Star rating' 
KSA_hotels.drop(columns=[column_to_drop], inplace=True)

KSA_hotels.to_csv('mod_KSA_hotels.csv', index=False)

In [49]:
########## you have to write documantation about what you did in 1.2 ###########

1.3 Data Processing

In [54]:
import pandas as pd

# Read the CSV file
KSA_hotels = pd.read_csv('mod_KSA_hotels.csv')

# Preprocessing 'Score' column into float 
KSA_hotels['Score'] = KSA_hotels['Score'].str.slice(9).astype(float)

# Preprocessing 'Number of reviews' column into int
KSA_hotels['Number of reviews'] = KSA_hotels['Number of reviews'].str.slice(stop=-7).str.replace(',','').astype(float)
KSA_hotels['Number of reviews'].fillna(0, inplace=True)  # Fill NaN values with 0
KSA_hotels['Number of reviews'] = KSA_hotels['Number of reviews'].astype(int)  # Convert to integer

# Preprocessing 'Price' column into int
KSA_hotels['Price'] = KSA_hotels['Price'].str.slice(3).str.replace(',','').astype(int)

print(KSA_hotels)


                                     Hotel Name        Address  Score  \
0                        Malfa Hotel Apartments  Qal'at Bishah    8.1   
1                        Intour Al Khafji Hotel      Al Khafji    7.7   
2                                Braira Al Ahsa        Al Ahsa    8.9   
3                   الراحة بلازا للشقق المفروشة       Sharurah    7.1   
4                                  Boudl Majmaa     Al Majmaah    8.1   
..                                          ...            ...    ...   
670                   Raoum Inn Khafji Corniche      Al Khafji    8.3   
671  Ramada Hotel & Suites by Wyndham Al Qassim  Al Bukayriyah    8.9   
672                          Makarim Palm Hotel         Rabigh    6.9   
673                      Malfa Hotel Apartments  Qal'at Bishah    8.1   
674                      Intour Al Khafji Hotel      Al Khafji    7.7   

     Number of reviews  Price                               Room type  \
0                  823   1250         Twin Room wi

In [51]:
# Preprocessing 'Room view', 'Swimming pool', 'Restaurant', 'Parking', 'Wheelchair accessibility' by encoding them into 0s and 1s

KSA_hotels['Room view'] = KSA_hotels['Room view'].replace({'yes': 1, 'no': 0})
KSA_hotels['Swimming pool'] = KSA_hotels['Swimming pool'].replace({'yes': 1, 'no': 0})
KSA_hotels['Restaurant'] = KSA_hotels['Restaurant'].replace({'yes': 1, 'no': 0})
KSA_hotels['Parking'] = KSA_hotels['Parking'].replace({'yes': 1, 'no': 0})
KSA_hotels['Wheelchair accessibility'] = KSA_hotels['Wheelchair accessibility'].replace({'yes': 1, 'no': 0})

print(KSA_hotels)

                                     Hotel Name        Address  Score  \
0                        Malfa Hotel Apartments  Qal'at Bishah    8.1   
1                        Intour Al Khafji Hotel      Al Khafji    7.7   
2                                Braira Al Ahsa        Al Ahsa    8.9   
3                   الراحة بلازا للشقق المفروشة       Sharurah    7.1   
4                                  Boudl Majmaa     Al Majmaah    8.1   
..                                          ...            ...    ...   
670                   Raoum Inn Khafji Corniche      Al Khafji    8.3   
671  Ramada Hotel & Suites by Wyndham Al Qassim  Al Bukayriyah    8.9   
672                          Makarim Palm Hotel         Rabigh    6.9   
673                      Malfa Hotel Apartments  Qal'at Bishah    8.1   
674                      Intour Al Khafji Hotel      Al Khafji    7.7   

    Number of reviews      Price                               Room type  \
0         823 reviews  SAR 1,250         Twin R

In [ ]:
######### you have to write documentation about what you have did in 1.3 #################

2 EDA

In [ ]:
##task1

In [ ]:
##task2

In [ ]:
##task3

In [ ]:
##task4